## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [2]:
df = pd.read_csv('Sites list (2).xlsx - Sheet1.csv')

In [3]:
df.columns

Index(['Type', 'Address', 'Owner', 'Current Use', 'Zoning',
       'Commercial Overlay', 'Max Units', 'Proposed Units', 'Affordability',
       'MBPO Comments'],
      dtype='object')

In [4]:
df = df.dropna(subset='Address')

## Google Maps API Geolocater Setup

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
df['geo_address'] = df['Address'] + ' Manhattan, New York'

In [7]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Correction Section

In [9]:
df.columns

Index(['Type', 'Address', 'Owner', 'Current Use', 'Zoning',
       'Commercial Overlay', 'Max Units', 'Proposed Units', 'Affordability',
       'MBPO Comments', 'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [10]:
df = df.fillna('Not Provided')

## Add colors to DF

In [11]:
df['Color'] = ''
df.loc[df['Type'] == 'Public', 'Color'] = 'Purple'
df.loc[df['Type'] == 'Private', 'Color'] = 'Green'
df.loc[df['Type'] == 'Mixed', 'Color'] = 'Orange'

## HTML Popup Formatter

In [12]:
def popup_html(row):
    Address = row['Address']
    Type = row['Type']
    Owner = row['Owner']
    Current_Use = row['Current Use']
    Zoning = row['Zoning']
    Commercial_Overlay = row['Commercial Overlay']
    Max_Units = row['Max Units']
    Proposed_Units = row['Proposed Units']
    Affordability = row['Affordability']
    Prez_comments = row['MBPO Comments']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Type: </strong>{}'''.format(Type) + '''<br>
    <strong>Owner: </strong>{}'''.format(Owner) + '''<br>
    <strong>Current Use: </strong>{}'''.format(Current_Use) + '''<br>
    <strong>Zoning: </strong>{}'''.format(Zoning) + '''<br>
    <strong>Commercial Overlay: </strong>{}'''.format(Commercial_Overlay) + '''<br>
    <strong>Max Units: </strong>{}'''.format(Max_Units) + '''<br>
    <strong>Proposed Units: </strong>{}'''.format(Proposed_Units) + '''<br>
    <strong>Affordability: </strong>{}'''.format(Affordability) + '''<br>
    <strong>Comments: </strong>{}'''.format(Prez_comments) + '''
    </html>
    '''
    return html

In [13]:
df.columns

Index(['Type', 'Address', 'Owner', 'Current Use', 'Zoning',
       'Commercial Overlay', 'Max Units', 'Proposed Units', 'Affordability',
       'MBPO Comments', 'geo_address', 'geocoded', 'lat', 'lon', 'Color'],
      dtype='object')

## Map Maker

In [14]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10.5,tiles=None)


### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Manhattan Development Opportunities")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        color=row['Color'],
        fill_color=row['Color'],
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)


lgd_txt = '<span style="color: {col};">{txt}</span>'

### Create legend ###
for i, r in df.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format(txt=r['Type'] + ' ' + '= ' + r['Color'],col=r['Color']))
    m.add_child(fg)
    
folium.TileLayer('CartoDBpositron',control=False).add_to(m)    

folium.map.LayerControl('topleft', collapsed=False).add_to(m)
    
# Get the HTML content of the legend element
legend_html = m._parent.get_root().html.render()

# Replace the checkbox elements with an empty string
m._parent.get_root().header.add_child(folium.Element('''
    <style>
    input[type=checkbox] {
        display: none !important;
    }
    </style>
'''))

# Update the content of the legend element
m._parent.get_root().html.content = legend_html

# Display map
m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/manhattan_dev_map_01022023
